In [75]:
# !pip install tensorflow==2.3.0
# !pip install keras
# !pip install keras-rl2

In [1]:
from gym import Env
from gym.spaces import Discrete, Box, Dict
import pandas as pd
import numpy as np
import random 
import time

In [2]:
# Create a virtual environment actions
def reset():
    global P, M, It, s
    dummy_array = np.zeros(shape=(P,8))
    df = pd.DataFrame(dummy_array,columns=['x','y','Day','Susceptible','Exposed','Infectious','Recovered','GG'])
    df = df.astype({'x':int,'y':int,'Day':int,'Susceptible':bool,'Exposed':int,'Infectious':int,'Recovered':bool,'GG':bool})
    df['Susceptible'] = True
    #Appending infectious population in 
    dfupdate=df.sample(M)
    dfupdate['Infectious'] = np.random.randint(1,It, size=len(dfupdate))
    dfupdate['Susceptible'] = False
    df.update(dfupdate)
    update_list = dfupdate.index.tolist() 
    #Dispersing people randomly among grid
    df['x'] = np.random.randint(0,s, size=len(df))
    df['y'] = np.random.randint(0,s, size=len(df))

    return df

def update_pos(p, df):
    global S
    df.loc[p,'x'] = max(min(df.loc[p,'x']+random.choice(range(-1,2)),s),0) #make valid movements in the grid
    df.loc[p,'y'] = max(min(df.loc[p,'y']+random.choice(range(-1,2)),s),0) 
    
def coor_around(p, df):
    return [(df.loc[p, 'x'] + a, df.loc[p, 'y'] + b) for a in range(-1,2) for b in range(-1, 2)]

def one_day(df, action=0):
    # start_time = time.time()
    global P, M, It, S, death_rate, expose_rate
    policy_match = {0: 1, 1: 0.75, 2: 0.25}  # assign action to policy
    moves_under_policy = int(round(Mt * policy_match[action], 0))

    df_infectious = df.loc[(df['Infectious'] > 0)]
    df_infectious = df_infectious[['x', 'y']]

    for mt in range(moves_under_policy):
        for index, person in df.iterrows():

            if not person['GG']:  # If the person is not dead

                new_move_x = random.choice(range(-1, 2))
                new_move_y = random.choice(range(-1, 2))

                person['x'] = max(min(person['x'] + new_move_x, s), 0)
                person['y'] = max(min(person['y'] + new_move_y, s), 0)

                df.iat[index, 0] = int(person['x'])
                df.iat[index, 1] = int(person['y'])

                if index in df_infectious.index:  # assigning whats in person (row) to df_infectious at the correct index
                    df_infectious.at[index, 'x'] = person['x']
                    df_infectious.at[index, 'y'] = person['y']

                if (person['Infectious'] > 0) and (person['Recovered'] == False):  # If a person is in infectious state
                    if person['Infectious'] - random.choice(range(0, 7)) >= It:  # If the infectious days are completed
                        if random.choice(range(0, death_rate)) > (
                                death_rate - 2):  # If the person dies(with probability distribution 1:4)
                            df.at[index, 'Infectious'] = 0
                            if index in df_infectious.index:
                                df_infectious.drop([index])

                            df.at[index, 'GG'] = True  # Kill the person
                        else:  # If the person survives
                            df.at[index, 'Infectious'] = 0
                            if index in df_infectious.index:
                                df_infectious.drop([index])
                            df.at[index, 'Recovered'] = True  # Recover the person
                    elif mt + 1 == moves_under_policy:
                        if person['Infectious'] == 0.5:
                            df.at[index, 'Infectious'] = 1
                        else:
                            df.at[index, 'Infectious'] = person['Infectious'] + 1  # Increase the infectious day counter
                        
                        
                        # print(f'No. {index} infected {person.Infectious} in day {d} at {mt}')
                                
                
                elif person['Exposed'] > 0:  # If a person is in exposed state
                    
                    if (person['Exposed'] - random.choice(range(0, 10))) >= Et:  # If the person has reached the exposed day limit?  7
                        
                        
                        df.at[index, 'Exposed'] = 0
                        df.at[index, 'Infectious'] = 0.5 if mt+1 != moves_under_policy else 1  # Increase the infectious day counter, now the person is infectious
                        df_infectious.append(person)
                    elif mt + 1 == moves_under_policy: # At the end of the day
                        if person['Exposed'] == 0.5:
                            df.at[index, 'Exposed'] = 1
                        else:
                            df.at[index, 'Exposed'] = person['Exposed'] + 1  # Increase the exposed day counter
                        
                        
                elif person['Susceptible']:  # If the person is in susceptible state
                    x_temp = int(person['x'])
                    df_xtemp = df_infectious[['x']].to_numpy()
                    if (x_temp in df_xtemp) or ((x_temp - 1) in df_xtemp) or ((x_temp + 1) in df_xtemp):
                        y_temp = int(person['y'])
                        df_ytemp = df_infectious[['y']].to_numpy()
                        if (y_temp in df_ytemp) or ((y_temp - 1) in df_ytemp) or ((y_temp + 1) in df_ytemp):
                            if random.choice(range(0, expose_rate)) > (expose_rate - 2):
                                df.at[index, 'Exposed'] = 0.5 if mt+1 != moves_under_policy else 1
                                df.at[index, 'Susceptible'] = False
    return df 



def economy_gain(df):
    economy_gain = len(df[(df.GG == False) & (df.Infectious == 0)]) * round(random.uniform(0.8,1), 2)
    return economy_gain

def current_state(df):
    global economy
    active_cases = len(df.loc[df['Infectious'] > 0])
    new_inf = len(df.loc[df['Infectious'] == 1])
    recovered = len(df.loc[df['Recovered'] == True])
    gg = len(df.loc[df['GG'] == True])
    reproduction_rate = len(df.loc[df['Infectious'] == 1]) / len(df.loc[df['Infectious'] > 1]) if len(df.loc[df['Infectious'] > 1]) > 0 else 0
    economy = economy + economy_gain(df)
        
    return np.array([active_cases, new_inf, recovered, gg, reproduction_rate, economy])



In [3]:
#Inputs
s = 200 #size of the grid
N = 1000 #size of population
M = round(N * 0.007) #Number of infectious population
Et = 2 #Number of days staying exposed
It = 21 #Number of days staying infectious
Mt = 8 #Number of daily movements
D = 100 #Number of days
death_rate = 100
expose_rate = 10

#Initialization
S = N - M #Susceptible population
E = 0 #Exposed population
I = M #Number of infectious population 
R = 0 #Recovered population
P = S + E + I + R #Total population
economy = 0 #Daily economic transaction

policy_match = {0: 1, 1:0.75, 2:0.25} # assign action to policy

In [4]:
def_observation_space = Box(low = np.array([0,0,0,0,0,0]), high = np.array([P,P,P,P,1,P*D*Mt], dtype = int))
##[active_cases, new_inf, recovered, gg, reproduction_rate, economy]
# Create the virtual environment for RL
class CoronaPolicy(Env):
    def __init__(self):
        self.action_space = Discrete(3)
        
        self.observation_space = def_observation_space # Box(low = 0, high = P, shape = (5,1), dtype = int )
        # Dict(recovered=Discrete(P+1), sus=Discrete(P+1), exposed=Discrete(P+1),inf=Discrete(P+1),gg=Discrete(P+1))
        
        self.df = reset()
        
        self.state = np.array(current_state(self.df))
        
        self.day = 0
        
    def step(self, action):
        
        self.df = one_day(self.df, action)
        
        self.state = current_state(self.df)
        
        self.day = self.day + 1
        
        reward = economy_gain(self.df)
        
        if self.day <= D:
            done = False
        else:
            done = True
            
        info = {}
        
        return self.state, reward, done, info
    
    def render(self):
        pass
    
    def reset(self):
        self.observation_space = def_observation_space
        
        self.df = reset()
        
        self.state = current_state(self.df)
        
        self.day = 0
        
        return self.state
        

C:\Users\Asus\anaconda3\lib\site-packages\gym\logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [6]:
env = CoronaPolicy()

# Deep Learning Model with Keras

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [8]:
def build_model(states, actions):
    model = Sequential()
    model.add(Dense(32, activation = 'relu',input_shape = states))
    model.add(Dense(64, activation = 'relu'))
    model.add(Dense(actions, activation = 'linear'))
    return model

In [14]:
del model

In [15]:
states = env.observation_space.shape
actions = env.action_space.n
model = build_model(states, actions)
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 32)                224       
_________________________________________________________________
dense_4 (Dense)              (None, 64)                2112      
_________________________________________________________________
dense_5 (Dense)              (None, 3)                 195       
Total params: 2,531
Trainable params: 2,531
Non-trainable params: 0
_________________________________________________________________


# Build Agent with Keras-RL

In [16]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy, EpsGreedyQPolicy, GreedyQPolicy
from rl.memory import SequentialMemory

In [17]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy() # ?
    memory = SequentialMemory(limit = 1000, window_length = 1)
    dqn = DQNAgent(model = model, memory = memory, policy = policy, 
                   nb_actions = actions, nb_steps_warmup = 10, target_model_update = 2000) # target_model_update
    return dqn

In [ ]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr = 1e-3), metrics = ['mae'])
dqn.fit(env, nb_steps = 2000, visualize = False, verbose = 1  )

Training for 2000 steps ...
Interval 1 (0 steps performed)
   11/10000 [..............................] - ETA: 22:20:59 - reward: 866.0264

C:\Users\Asus\anaconda3\lib\site-packages\rl\memory.py:40: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


  925/10000 [=>............................] - ETA: 8:51:22 - reward: 708.9120

In [35]:
# Test the agent
import tensorflow as tf
df = reset()
economy = 0
states = []
for day in range(0, 200):
    state = current_state(df)
    states.append(state)
    
    state = tf.reshape(state, [1, 6])
    
    prediction = model.predict(state, steps = 1)
    action_by_agent = np.argmax(prediction)
    df = one_day(df, action = action_by_agent)
    gain = economy_gain(df)
    economy += gain
    print(f"Day {day}: take action {action_by_agent}, total_reward: {economy}. {prediction}")

Day 0: take action 1, total_reward: 1866.84. [[152.96213 154.71457 151.25412]]
Day 1: take action 1, total_reward: 3703.8899999999994. [[388.22726 400.1671  392.26306]]
Day 2: take action 1, total_reward: 5479.54. [[636.48315 655.56494 645.30273]]
Day 3: take action 1, total_reward: 7191.2699999999995. [[887.29724 914.4976  897.8619 ]]
Day 4: take action 1, total_reward: 8906.83. [[1133.044  1167.9805 1146.1385]]
Day 5: take action 1, total_reward: 10627.07. [[1370.5613 1412.6481 1387.3358]]
Day 6: take action 1, total_reward: 12388.849999999999. [[1591.2097 1639.5162 1612.8441]]
Day 7: take action 1, total_reward: 14195.749999999998. [[1876.2662 1934.5146 1897.5265]]
Day 8: take action 1, total_reward: 16005.159999999998. [[2161.5686 2228.5662 2181.8481]]
Day 9: take action 1, total_reward: 17578.43. [[2421.5408 2495.3877 2442.5544]]
Day 10: take action 1, total_reward: 19254.87. [[2779.9185 2851.3499 2788.36  ]]
Day 11: take action 1, total_reward: 20908.45. [[3165.1592 3227.4407 315

Day 93: take action 2, total_reward: 130022.48. [[14849.539 15198.594 15296.567]]
Day 94: take action 2, total_reward: 131909.18. [[15089.007 15442.162 15540.649]]
Day 95: take action 2, total_reward: 133795.88. [[15344.439 15701.968 15801.003]]
Day 96: take action 2, total_reward: 135533.63. [[15578.589 15940.127 16039.664]]
Day 97: take action 2, total_reward: 137231.66. [[15808.743 16174.22  16274.252]]
Day 98: take action 2, total_reward: 139138.22. [[16053.534 16423.201 16523.758]]
Day 99: take action 2, total_reward: 140846.18000000002. [[16290.344 16664.064 16765.13 ]]
Day 100: take action 2, total_reward: 142574.0. [[16523.16  16900.87  17002.434]]
Day 101: take action 2, total_reward: 144242.24. [[16753.316 17134.963 17237.021]]
Day 102: take action 2, total_reward: 146009.78. [[16990.123 17375.822 17478.389]]
Day 103: take action 2, total_reward: 147717.74. [[17214.957 17604.51  17707.557]]
Day 104: take action 2, total_reward: 149455.49. [[17437.129 17830.488 17934.012]]
Day

Day 184: take action 2, total_reward: 291603.44. [[36477.57  37185.723 37332.55 ]]
Day 185: take action 2, total_reward: 293410.7. [[36731.86 37446.64 37592.23]]
Day 186: take action 2, total_reward: 295168.31. [[36959.664 37680.387 37824.86 ]]
Day 187: take action 2, total_reward: 296856.41000000003. [[37186.14  37912.766 38056.133]]
Day 188: take action 2, total_reward: 298604.09. [[37431.156 38164.176 38306.34 ]]
Day 189: take action 2, total_reward: 300272.33. [[37645.72 38384.33 38525.44]]
Day 190: take action 2, total_reward: 302109.38. [[37892.06  38637.1   38777.008]]
Day 191: take action 2, total_reward: 303777.62. [[38117.21  38868.117 39006.93 ]]
Day 192: take action 2, total_reward: 305495.51. [[38338.395 39095.066 39232.797]]
Day 193: take action 2, total_reward: 307203.47000000003. [[38575.46  39338.324 39474.895]]
Day 194: take action 2, total_reward: 309000.80000000005. [[38816.508 39585.65  39721.043]]
Day 195: take action 2, total_reward: 310827.92000000004. [[39046.9

In [18]:
model.save("model_ann_4_6_states_BoltzmannQPolicy_tmu2000")

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: model_ann_4_6_states_BoltzmannQPolicy_tmu2000\assets
